In [180]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import pytz

In [167]:
df = pd.read_excel("/mnt/c/Users/riguy/Downloads/AmITooSweet.xlsx")

In [168]:
df = df[['Date', 'Time', 'Type', 'Units', 'Notes']]

In [169]:
def fix_timezones(s):
    if pd.isna(s):
        return s
    s = s.lower()
    if 'time zone' in s:
        return s.replace('time zone', '').strip().upper()
    return np.nan

df.Date = df.Date.interpolate(method='pad')

df["timezone"] = df.Notes.apply(fix_timezones)
df.at[0, 'timezone'] = 'PST' # fix first value

df['had_timezone'] = pd.isna(df.timezone) == False

df.timezone = df.timezone.interpolate(method='pad')

In [170]:
df.head()

,Date,Time,Type,Units,Notes,timezone,had_timezone
0,2022-11-07,19:57:00,Lantis,10.0,New pen,PST,True
1,2022-11-08,19:52:00,Lantis,10.0,NaN,PST,False
2,2022-11-09,19:26:00,Lantis,10.0,Hurt. 4mm needle. Pulled out early?,PST,False
3,2022-11-10,19:57:00,Lantis,10.0,NaN,PST,False
4,2022-11-11,19:12:00,Lantis,11.0,5mm needle,PST,False


In [171]:
df_tz_points

,timezone,timestamp
0,PST,2022-11-07 19:57:00-08:00
20,EST,2022-11-18 20:48:00-05:00


In [172]:
tz_correct = {
    'PST': 'America/Los_Angeles'
}

def build_timestamp(row):
    s = f"{row['Date'].strftime('%Y-%m-%d')}T{row['Time'].isoformat()}"
    tz = pytz.timezone(tz_correct.get(row['timezone'], row['timezone']))
    d = datetime.strptime(s, '%Y-%m-%dT%H:%M:00')
    return tz.localize(d)
    
df['timestamp'] = df.apply(build_timestamp, axis=1)

In [173]:
df_tz_points = df[df.had_timezone]
df_tz_points = df_tz_points[['timezone', 'timestamp']]
df_tz_points

,timezone,timestamp
0,PST,2022-11-07 19:57:00-08:00
20,EST,2022-11-18 20:48:00-05:00


In [175]:
df = df.drop(columns='had_timezone')

In [182]:
# This is a sanity check that our values are in order. 
# I've caught a few data errors this way:
#  * wrong TZ info
#  * bad date format
#  * forgot to use 24 time.
assert df.timestamp.diff().min() > timedelta(0)

AssertionError: 

In [177]:
df

,Date,Time,Type,Units,Notes,timezone,timestamp
0,2022-11-07,19:57:00,Lantis,10.0,New pen,PST,2022-11-07 19:57:00-08:00
1,2022-11-08,19:52:00,Lantis,10.0,NaN,PST,2022-11-08 19:52:00-08:00
2,2022-11-09,19:26:00,Lantis,10.0,Hurt. 4mm needle. Pulled out early?,PST,2022-11-09 19:26:00-08:00
3,2022-11-10,19:57:00,Lantis,10.0,NaN,PST,2022-11-10 19:57:00-08:00
4,2022-11-11,19:12:00,Lantis,11.0,5mm needle,PST,2022-11-11 19:12:00-08:00
5,2022-11-12,19:40:00,lantis,12.0,NaN,PST,2022-11-12 19:40:00-08:00
6,2022-11-13,19:45:00,Lantis,12.0,4mm,PST,2022-11-13 19:45:00-08:00
7,2022-11-14,19:46:00,Lantis,13.0,had overnight low,PST,2022-11-14 19:46:00-08:00
8,2022-11-15,19:53:00,lantis,12.0,NaN,PST,2022-11-15 19:53:00-08:00
9,2022-11-16,20:01:00,Lantis,11.0,kept going high next day,PST,2022-11-16 20:01:00-08:00
